Общее пояснение: данный файл содержит результаты ручной разметки текстов; одна строка = результаты работы одного ассесора по одному тексту; тексты повторяются, но их оценки уникальны. Конечная задача: обучить классификатор (в тесте не ставится)
Вкладка «куда вставлять» содержит файл, в котором неправильно заполнены столбцы с X по AD (отношение … опасный). Вкладка «что вставлять» содержит правильные значения этих столбцов. Неправильные значения надо заменить на правильные. Однако есть ряд проблем.

### Проблема 1.
Каждая строка вкладки "куда вставлять" имеет идентификационный номер, но номера повторяются. Если они повторяются, строки различаются по столбцу «логин». Во вкладке с правильными значениями столбцов X-AD ("что вставлять") сохранены IDs, но не сохранены значения столбца «логин». Таким образом, соотнести однозначно строки вкладки «что вставлять» и сторки вкладки «куда вставлять» невозможно. Надо брать ID из вкладки «что вставлять», искать его первое попавшееся упоминание во вкладке «куда вставлять» и, собственно, вставлять. Затем нужно взять во вкладке «что вставлять» следующую строку  с тем же ID и найти во вкладке «куда вставлять» СЛЕДУЮЩЕЕ (но не то же самое!) упоминание этого ID. Важно не запихать второй раз правильные значения в ту строку вкладки «куда вставлять», которую уже один раз заменили. А то в одной строке значения поменяем дважды, а в другой – ни разу.


### Проблема 2. 
Во вкладке «что вставлять» явно есть лишние данные, т.к. там почти в два раза больше строк. Те строки из вкладки «что вставлять», которые окажутся не востребованными и никуда не вставятся, записать в отдельный файл.


### Проблема 3. 
не исключено, если во вкладке «что вставлять» будет не хватать строк, которые есть во вкладке «что вставлять». Такой нехватки может и не быть. Но если все же во вкладке «куда вставлять» останутся строки, в которых ни разу не были заменены столбцы X-AD, такие строки надо сохранить в третий файл.


### NB:
обратите внимание, во вкладке "куда вставлять" столбцы не имеют уникальных заголовков. Так, заголовки "отношение", "опасный" и т.д. повторяются. Нас интересуют именно столбцы X-AD. В csv  и других форматах столбцы не будут иметь идентификаторов  типа X и AD, поэтому важно не перепутать между собой столбцы с одинаковыми заголовками. Нас интересует то "отношение", которые самое первое (самый левый из всех столбцов с заголовком "отношение"). Аналогично для других пяти столбцов.
результат представить в формате xls или csv (разделители целой части числа - запятые). Приложить код.


In [2]:
import pandas as pd
import numpy as np

In [159]:
where_all = pd.read_csv('тест датамйнерам.csv', low_memory=False)

In [118]:
#маска определяющаяя где у нас вообще есть id
id_mask = np.logical_not(where_all.id.isnull().values)
print id_mask.sum()


# выделем осмысленную часть датасета
where = where_all.iloc[id_mask]

2029


In [119]:
what = pd.read_csv('тест датамйнерам что.csv', low_memory=False)

Будем ходить двумя указателями, так что нужно, чтобы массивы были отсортированы по индексу. Порядок сортировки довольно специфический, так что функция сравнения своя

In [120]:
def id_compare(id_1, id_2):
    if id_1.isdigit() and id_2.isdigit():
        return int(id_1) <= int(id_2)
    elif id_1.isdigit():
        return True
    elif id_2.isdigit():
        return False
    else:
        return id_1 <= id_2

In [121]:
print all(map(lambda x : id_compare(x[0], x[1]), zip(what.id.values[:-1], what.id.values[1:])))
print all(map(lambda x : id_compare(x[0], x[1]), zip(where.id.values[:-1], where.id.values[1:]) ))

True
True


Оба датасета действительно отсортированы, дальше идем двумя указателями, что куда надо вставляем

In [122]:
i = 0
j = 0

unreplaced_where = pd.DataFrame(columns=where.columns)
unused_what = pd.DataFrame(columns=what.columns)

i_lim = where.shape[0]
j_lim = what.shape[0]

while i < i_lim and j < j_lim: 
    if i % 100 == 0:
        print 'i', i
        #print "j", j
    if where.loc[i, 'id'] == what.loc[j, 'id']:
        where.loc[i, what.columns] = what.iloc[j]
        i += 1
        j += 1
    elif id_compare(where.loc[i, 'id'], what.loc[j, 'id']):
        unreplaced_where.loc[i, where.columns] = where.iloc[i]
        i += 1   
    else:
        unused_what.loc[j, what.columns] = what.iloc[j]
        j += 1


i 0
i 0
i 100
i 200
i 300
i 400
i 400
i 500
i 500
i 500
i 600
i 700
i 800
i 900
i 900
i 900
i 1000
i 1000
i 1100
i 1200
i 1200
i 1300
i 1400
i 1500
i 1500
i 1600
i 1700
i 1700
i 1700
i 1700
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1800
i 1900
i 1900
i 1900
i 2000
i 2000
i 2000


Теперь проверим, дошли ли до конца, или остались строки, которые нужно куда-то сложить

In [124]:
print i, j
print i_lim, j_lim

2029 3344
2029 3344


Все на месте, кроме кусков из "куда вставлять", без id

Посмотрим глазами на то что получилось и сохраним 

In [130]:
unused_what.reset_index(inplace = True)
unused_what.head()

,index,id,Обобщение,Отношение,Агрессия,Солидар.,Низ/выс,Жертва?,Опасный?
0,0,2942,3,2,1,1,3,3,1
1,12,55368,2,2,1,1,3,3,1
2,16,58073,2,2,1,1,3,3,1
3,21,62855,1,2,1,2,3,3,1
4,22,62855,1,2,1,1,3,3,1


In [131]:
unreplaced_where

,id,Время,Этноним,Логин,Понятно?,Этноним.1,Негатив,Позитив,Конфликт,Поз.взаим.,...,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83


In [158]:
what.to_csv('what.csv')
where.to_csv('where.csv', float_format='%.0f')
unused_what.to_csv('unused_what.csv')

In [172]:
nn_mask = np.logical_not(where_all.iloc[where_all.id.isnull().values].isnull()).values

In [175]:
pd.DataFrame(where_all.iloc[where_all.id.isnull().values].values[nn_mask]).to_csv('rubish.csv')

# 2
найти все тексты, имеющие три и более оценки по полю "отношение"; вывести их количество

перекодировать значения поля "отношение":  pos=1, neutral=2, neg=3

посчитать среднее и округлить до ближайшего целого

посчитать распределения неокругленных и округленных значений

результат представить в виде трех таблиц xls или csv, разделители дробной части - запятые):
а. столбцы: Id, оригинальный текст, оценка 1, оценка 2, оценка 3, среднее, округленное среднее
б.  Таблица частот неокругленных средних
в. Таблица частот округленных средних
приложить код


In [97]:
where = pd.read_csv('where.csv')

#### Создаем маску тех id которые встречаются три или более раз, и выделяем их в отдельный список. 
Отношение у всех заполнено, так как мы заполняя таблицу в задании 1 не столкнулись ни с одной строкой которая была бы в where но не была бы в what

In [98]:
ids_mask = (where.id.value_counts() >= 3).values
target_ids = where.id.value_counts().index[ids_mask]

In [99]:
print 'Кол-во id к которым относится три или более записей:', len(target_ids)

Кол-во id к которым относится три или более записей: 122


#### Кол-во id к которым относится три или более записей: 122

Теперь сначала выделим интересные id в отдельную таблицу, у таблицы будет пять колонок, потому что это максимальное кол-во "Отношений" для одного id, id будут индексами таблицы. Затем найдем среднее значение "Отношениее" для каждого id, потом округлим и потом уже переименуем окончательные цифры в pos\neutral\neg 

In [100]:
target_data = pd.DataFrame(columns=['Отношение_0','Отношение_1','Отношение_2','Отношение_3','Отношение_4'])

column_id = 0
for i, row in where.iterrows():
    if row.id in target_ids:
        if row.id in target_data.index:
            column_id += 1
        else:
            column_id = 0
        
        column_name = 'Отношение_' + str(column_id)
        target_data.loc[row.id, column_name] = row['Отношение']

In [101]:
target_data.head()

,Отношение_0,Отношение_1,Отношение_2,Отношение_3,Отношение_4
93672,3,3,3,NaN,NaN
101899,2,3,1,NaN,NaN
115795,1,1,1,NaN,NaN
170997,3,3,3,NaN,NaN
235462,2,1,1,1,NaN


Так выглядит таблица, теперь посчитаем среднее. 

In [102]:
target_avrg = pd.DataFrame(target_data.sum(1).astype(float) / np.logical_not(target_data.isnull().values).sum(1), columns = ['Среднее'])

In [103]:
target_avrg.head()

,Среднее
93672,3.00
101899,2.00
115795,1.00
170997,3.00
235462,1.25


Округлим

In [104]:
target_avrg_int = pd.DataFrame(np.around(target_avrg))
target_avrg_int.columns = ['Округленное Среднее']
target_avrg_int.head()

,Округленное Среднее
93672,3
101899,2
115795,1
170997,3
235462,1


Переименуем

In [105]:
renamer_dict = {1 : 'pos',
                2 : 'neutral',
                3 : 'neg'}

In [106]:
target_avrg_renamed = target_avrg_int.replace(renamer_dict)
target_avrg_renamed.columns = ['Среднее Назв']

In [107]:
target_avrg_renamed.head()

,Среднее Назв
93672,neg
101899,neutral
115795,pos
170997,neg
235462,pos


In [108]:
target_final = pd.concat((target_data,target_avrg,target_avrg_int,target_avrg_renamed), axis = 1)

In [139]:
target_final.head()

,Отношение_0,Отношение_1,Отношение_2,Отношение_3,Отношение_4,Среднее,Округленное Среднее,Среднее Назв
93672,3,3,3,NaN,NaN,3.00,3,neg
101899,2,3,1,NaN,NaN,2.00,2,neutral
115795,1,1,1,NaN,NaN,1.00,1,pos
170997,3,3,3,NaN,NaN,3.00,3,neg
235462,2,1,1,1,NaN,1.25,1,pos


In [111]:
target_final.to_csv('Отношение_main.csv', decimal=',')

#### Посчитаем статистики

In [120]:
stats_avr = pd.DataFrame(target_final['Среднее'].value_counts())
stats_avr.columns = ['Кол-во']

In [134]:
stats_avr.sort_index()

,Кол-во
1.000000,24
1.250000,1
1.333333,16
1.500000,1
1.666667,13
2.000000,24
2.250000,1
2.333333,17
2.500000,2
2.666667,8


In [135]:
stats_avr_round = pd.DataFrame(target_final['Округленное Среднее'].value_counts())
stats_avr_round.columns = ['Кол-во']
stats_avr_round.sort_index()

,Кол-во
1,41
2,58
3,23


In [137]:
stats_avr.sort_index().to_csv('stats_avrg.csv', decimal = ',')
stats_avr_round.sort_index().to_csv('stats_avrg_round.csv')